# Exploring GCN #2

Fully connected graph, correlation as edge features.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pickle

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import spektral as sp

# Restore preprocessed datasets

In [3]:
%store -r np_timeseries

In [4]:
df_patients = pd.read_csv('patients-cleaned.csv', index_col=0)

In [5]:
X = np_timeseries
y = df_patients['target'].values

In [6]:
print(X.shape)
print(y.shape)

(190, 90, 400)
(190,)


In [7]:
# with open('granger-lambda=1-400-nopreprocessing.pickle', 'rb') as f:
#     gs = pickle.load(f)

# with open('covariance-400-nopreprocessing.pickle', 'rb') as f:
#     cov = pickle.load(f)

with open('correlation-400-nopreprocessing.pickle', 'rb') as f:
    corr = pickle.load(f)

In [8]:
# Parameters
samples = X.shape[0]            # Number of subjects.       | 190
N = regions = X.shape[1]        # Number of regions.        | 90
F = X.shape[2]                  # Features - timeseries.    | 400
S = 1                           # Edge - correlation        | 1
n_classes = 2                   # Control / patient.        | 2

## Training and testing datasets

In [26]:
# Train/test split
E_train, E_test, \
A_train, A_test, \
x_train, x_test, \
y_train, y_test = train_test_split(corr, np.ones((samples, N, N)), np.ones((samples)), y, test_size=0.2, shuffle=True, random_state=42)


In [35]:
# From `example-spectral`.
l2_reg = 0.0001            # Regularization rate for l2
learning_rate = 0.001     # Learning rate for Adam
epochs = 200              # Number of training epochs
batch_size = 4              # Batch size.

In [39]:
E_in.shape

TensorShape([None, 1])

In [40]:
# Model definition
X_in = tf.keras.layers.Input(shape=(1,))
A_in = tf.keras.layers.Input(shape=(None,), sparse=True)
E_in = tf.keras.layers.Input(shape=(1,))

X_1 = sp.layers.EdgeConv(32, activation="relu")([X_in, A_in, E_in])
X_2 = sp.layers.EdgeConv(32, activation="relu")([X_1, A_in, E_in])
X_3 = sp.layers.GlobalSumPool()([X_2])

output = tf.keras.layers.Dense(1, activation="sigmoid")(X_3)

In [41]:
# Build model
model = tf.keras.models.Model(inputs=[X_in, A_in, E_in], outputs=output)
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
edge_conv_11 (EdgeConv)         (None, 32)           64          input_22[0][0]                   
                                                                 input_23[0][0]        

In [42]:
#stopping_cb = tf.keras.callbacks.EarlyStopping(patience=es_patience, restore_best_weights=True)
lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50)

136.8

In [43]:
# Train model
model.fit(
    [x_train, A_train, E_train],
    y_train,
    # batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[lr_cb]
)

Epoch 1/200


AssertionError: in user code:

    c:\Users\janar\Documents\MFF\DiplomaThesis\BrainConnectivityMachineLearning\data\.venv\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\Users\janar\Documents\MFF\DiplomaThesis\BrainConnectivityMachineLearning\data\.venv\lib\site-packages\spektral\layers\convolutional\message_passing.py:83 call  *
        X, A, E = self.get_inputs(inputs)
    c:\Users\janar\Documents\MFF\DiplomaThesis\BrainConnectivityMachineLearning\data\.venv\lib\site-packages\spektral\layers\convolutional\message_passing.py:146 get_inputs  *
        assert K.ndim(E) == 2, 'E must have rank 2'

    AssertionError: E must have rank 2


## Only graph no node features

In [36]:
# Parameters
batch_size = 4  # Batch size
epochs = 200  # Number of training epochs
patience = 10  # Patience for early stopping
l2_reg = 0.001  # Regularization rate for l2
l2 = tf.keras.regularizers.L2(l2=l2_reg)

In [37]:
# Model definition
X_in = tf.keras.layers.Input(shape=(N, F))
A_in = tf.keras.layers.Input((N, N))

x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([X_in, A_in])
x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([x, A_in])
x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([x, A_in])

x = sp.layers.GlobalSumPool()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [38]:
model = tf.keras.models.Model(inputs=[X_in, A_in], outputs=outputs)
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "functional_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 90, 400)]    0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 90, 90)]     0                                            
__________________________________________________________________________________________________
graph_conv_8 (GraphConv)        (None, 90, 8)        3208        input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
graph_conv_9 (GraphConv)        (None, 90, 8)        72          graph_conv_8[0][0]   

In [39]:
# Train model
model.fit(
    [x_train, A_train],
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs
)

Epoch 1/200
39/39 [==============================] - 1s 15ms/step - loss: 19970.5059 - acc: 0.5163 - val_loss: 4865.2441 - val_acc: 0.4444
Epoch 2/200
39/39 [==============================] - 0s 4ms/step - loss: 4493.6113 - acc: 0.6078 - val_loss: 5716.4966 - val_acc: 0.2778
Epoch 3/200
39/39 [==============================] - 0s 5ms/step - loss: 1183.4435 - acc: 0.7451 - val_loss: 7570.2681 - val_acc: 0.5556
Epoch 4/200
39/39 [==============================] - 0s 4ms/step - loss: 1352.3500 - acc: 0.8039 - val_loss: 3181.0239 - val_acc: 0.4444
Epoch 5/200
39/39 [==============================] - 0s 5ms/step - loss: 343.5517 - acc: 0.8235 - val_loss: 4501.6235 - val_acc: 0.5000
Epoch 6/200
39/39 [==============================] - 0s 4ms/step - loss: 345.6089 - acc: 0.8301 - val_loss: 4866.2202 - val_acc: 0.4444
Epoch 7/200
39/39 [==============================] - 0s 4ms/step - loss: 510.2189 - acc: 0.7974 - val_loss: 5089.5698 - val_acc: 0.4444
Epoch 8/200
39/39 [=======================